In [1]:
using Statistics: mean
using Flux
using Flux:onehotbatch, onecold, crossentropy, throttle, @epochs
using CUDA

In [2]:
imgs = Flux.Data.MNIST.images()
labels = Flux.Data.MNIST.labels();

In [3]:
imgs[123]

In [4]:
labels[123]

2

In [5]:
X = hcat(float.(reshape.(imgs, :))...) |> gpu
Y = onehotbatch(labels, 0:9) |> gpu;

In [ ]:
test_imgs = Flux.Data.MNIST.images(:test)
test_labels = Flux.Data.MNIST.labels(:test)

test_X = hcat(float.(reshape.(test_imgs, :))...) |> gpu
test_Y = onehotbatch(test_labels, 0:9)  |> gpu

In [6]:
m = Chain(
Dense(28^2, 128, relu),
Dense(128, 64, relu),
Dense(64, 10),
softmax
) |> gpu

Chain(Dense(784, 128, relu), Dense(128, 64, relu), Dense(64, 10), softmax)

In [7]:
loss(x,y) = crossentropy(m(x), y)
opt = ADAM();

In [8]:
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y));

In [33]:
#dataset = [(X, Y)]
dataset = Flux.Data.DataLoader(X, Y, batchsize=32);

In [34]:
evalcb = () -> @show(loss(X, Y))

#48 (generic function with 1 method)

In [35]:
@epochs 3 Flux.train!(loss, params(m), dataset, opt, cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main D:\.julia\packages\Flux\goUGu\src\optimise\train.jl:136


loss(X, Y) = 0.026096469f0
loss(X, Y) = 0.06306351f0


┌ Info: Epoch 2
└ @ Main D:\.julia\packages\Flux\goUGu\src\optimise\train.jl:136


loss(X, Y) = 0.04055601f0


┌ Info: Epoch 3
└ @ Main D:\.julia\packages\Flux\goUGu\src\optimise\train.jl:136


In [36]:
accuracy(X, Y)

0.9887333333333334

In [37]:
test_imgs = Flux.Data.MNIST.images(:test)
test_labels = Flux.Data.MNIST.labels(:test)

test_X = hcat(float.(reshape.(test_imgs, :))...) |> gpu
test_Y = onehotbatch(test_labels, 0:9)  |> gpu

10×10000 Flux.OneHotMatrix{CuArray{Flux.OneHotVector,1}}:
 0  0  0  1  0  0  0  0  0  0  1  0  0  …  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  1  0  0  1  0  0  0  0  0  0  0     0  0  0  0  0  0  1  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  1  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0  …  1  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  1  0     0  1  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  1  0  0  1     0  0  0  0  1  0  0  0  0  0  0  0

In [18]:
accuracy(test_X, test_Y)

0.9476